In [1]:
import time
import numpy as np
from heavy_tail_observations import BothSideWeibullNoise, BothSideParetoNoise, BothSideFrechetNoise

from matplotlib import pyplot as plt
%matplotlib inline

In [23]:
scale=1.
mean=0.
p=1.5
noise_type = 'pareto'

dim = 5
weight = np.random.randn(dim)
weight = weight/np.sqrt(weight.dot(weight))
    
get_mean = lambda x: x.dot(weight)
if noise_type == 'weibull':
    weibull_noise = BothSideWeibullNoise(alpha=p+0.1,scale=scale,mean=mean,p=p)
    get_observation = lambda x: x.dot(weight) + weibull_noise.sample()
elif noise_type == 'pareto':
    pareto_noise = BothSideParetoNoise(alpha=p+0.1,scale=scale,mean=mean,p=p)
    get_observation = lambda x: x.dot(weight) + pareto_noise.sample()
elif noise_type == 'frechet':
    frechet_noise = BothSideFrechetNoise(alpha=p+0.1,scale=scale,mean=mean,p=p)
    get_observation = lambda x: x.dot(weight) + frechet_noise.sample()
    
K = 10
T = 10000

mom

In [24]:
def median_of_means_order_OFU(d, c, epsilon, delta, lamb, S, T, K):
    # initialization
    k = int(np.ceil(24 * np.log(epsilon * T / delta)))
    N = int(T // k)
    V = lamb * np.eye(d)
    theta_star = np.zeros(d)
    x = np.zeros((N, d))
    y = np.zeros((N, k))
    beta = S
    
    # algorithm
    for n in range(N):
        # pick D_n
        D = np.zeros((K, d))
        for i in range(K):
            D[i] = np.random.normal(size=d)
        
        # line 4 : select an arm
        # norm?
        dot = np.zeros(K)
        for i in range(K):
            lagrange = beta / np.linalg.norm(D[i])
            theta_max = (lagrange * D[i]) + theta_star
            dot[i] = np.dot(D[i], theta_max)
        x[n] = D[np.argmax(dot)]
        
        # line 5 : observe payoffs
        for i in range(k):
            y[n][i] = get_observation(x[n])
        
        # line 6 : update V
        for i in range(d):
            for j in range(d):
                V[i][j] = V[i][j] + x[n][i] * x[n][j]
                
        # line 7 : calculate LSE for the j-th group
        theta_hat_n = np.zeros((k, d))
        for j in range(k):
            sum = np.zeros(d)
            for i in range(n + 1):
                sum += y[i][j] * x[i]
            theta_hat_n[j] = np.dot(np.linalg.inv(V), sum)
            
        # line 8 : find median
        r = np.zeros(k)
        for j in range(k):
            r_med = np.zeros(k - 1)
            for s in range(k):
                theta = theta_hat_n[j] - theta_hat_n[s]
                if s < j:
                    r_med[s] = np.sqrt(np.dot(np.dot(theta.T, V), theta))
                elif s > j:
                    r_med[s - 1] = np.sqrt(np.dot(np.dot(theta.T, V), theta))
            
            r[j] = np.median(r_med)
        
        # line 9 : take median of means of estimates
        k_star = np.argmin(r)

        # line 10 : compute beta
        beta = 3 * (((9 * d * c) ** (1 / (1 + epsilon))) * ((n + 1) ** ((1 - epsilon) / (2 * (1 + epsilon)))) + (lamb ** (1 / 2)) * S)
        
        # line 11 : update the confidence region
        theta_star = theta_hat_n[k_star]
        
    return theta_star

In [51]:
c=1000.
epsilon=p-1
delta=0.01
lamb=1.
S=1.
d=dim

# initialization
k = int(np.ceil(24 * np.log(epsilon * T / delta)))
N = int(T // k)
V = lamb * np.eye(d)
Vinv = np.linalg.inv(V)
theta_star = np.zeros(d)
x = np.zeros((N, d))
y = np.zeros((N, k))
beta = S
cum_regret = 0

# algorithm
for n in range(N):
    # pick D_n
    D = np.random.normal(size=(K,d))

    # line 4 : select an arm
    scores = D.dot(theta_star) + beta*np.sqrt(np.diag(D.dot(Vinv.dot(D.T))))
    x[n] = D[np.argmax(scores)]

    # line 5 : observe payoffs
    y[n] = [get_observation(x[n]) for _ in range(k)]

    # line 6 : update V
    V = V + x[n][:,np.newaxis]*x[n][np.newaxis,:]
    Vinv = np.linalg.inv(V)
      
    # line 7 : calculate LSE for the j-th group
    thetas = Vinv.dot(x[:(n+1),:].T.dot(y[:(n+1),:])).T
    
    # line 8 : find median
    r = []
    for j in range(k):
        r_med = []
        for s in range(k):
            delta_theta = thetas[j] - thetas[s]
            r_med.append(np.dot(np.dot(delta_theta.T, V), delta_theta))
        r.append(np.median(r_med))
    
    # line 9 : take median of means of estimates
    k_star = np.argmin(r)

    # line 10 : compute beta
    beta = 3 * (((9 * d * c) ** (1 / (1 + epsilon))) * ((n + 1) ** ((1 - epsilon) / (2 * (1 + epsilon)))) + (lamb ** (1 / 2)) * S)

    # line 11 : update the confidence region
    theta_star = thetas[k_star]
    
    # line 12 : Regret
    cum_regret += np.max(get_mean(D)) - get_mean(x[n])
    print(cum_regret/(n+1))

3.4912713094701946
2.3640852029318085
2.444455401554087
1.8333415511655653
1.4666732409324523
1.2222277007770435
1.1420446850114527
1.2943697260731997
1.4921904194035542
1.544376314893216
1.4819017124557305
1.3865676269863634
1.489433237217444
1.4949698545311614
1.3953051975624173
1.3279321896636118
1.3226381076598352
1.3715278895384277
1.2993422111416684
1.2536300463264767
1.3561979151629235
1.2945525553827906
1.3237239093157351
1.3269003673996311
1.348180587724794
1.2963274881969176
1.2673857964093145
1.3369598091668418
1.2908577467817781
1.3304044808915994
1.4046072940807721


turncation